In [ ]:
# batch_first=True로 설정하면 입력을 (batch_size, sequence_length, input_size) 이렇게 받는다

In [ ]:
class My_LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length) : 
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first = True)
        self.layer_1 = nn.Linear(hidden_size, 256)
        self.layer_2 = nn.Linear(256,256)
        self.layer_3 = nn.Linear(256,128)
        self.layer_out = nn.Linear(128, num_classes)
        self.relu = nn.ReLU() 

    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #Hidden State
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #Internal Process State
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn.view(-1, self.hidden_size) # 1차원으로 변환해줌

        out = self.relu(hn) 
        out = self.layer_1(out) 
        out = self.relu(out) 
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.relu(out)
        out = self.layer_out(out) 

        return out

In [ ]:
# Code Mai
num_epochs = 10000
learning_rate = 0.001
input_size = int(len(x.columns))
hidden_size = 2 # number of features in hidden state
num_layers = 1
num_classes = int(len(y.columns))
LSTM_Jun = LSTM_Jun(num_classes, input_size, hidden_size, num_layers, train_x_tensor_final.shape[1]).to(device)
loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(LSTM_Jun.parameters(), lr = learning_rate)
for epoch in range(num_epochs) : 
    outputs = LSTM_Jun.forward(train_x_tensor_final.to(device))
    optimizer.zero_grad()
    loss = loss_function(outputs, train_y_tensor.to(device))
    loss.backward()
    optimizer.step() # improve from loss = back propagation
    if epoch % 200 == 0 :
        print("Epoch : %d, loss : %1.5f" % (epoch, loss.item()))
# Estimated Value
test_predict = LSTM_Jun(train_x_tensor_final.to(device)) #Forward Pass
predict_data = test_predict.data.detach().cpu().numpy() #numpy conversion
predict_data = minmax.inverse_transform(predict_data) #inverse normalization(Min/Max)

# Real Value
real_data = train_y_tensor.data.numpy() # Real value
real_data = minmax.inverse_transform(real_data) #inverse normalization
#Figure

plt.figure(figsize = (10,6)) # Plotting
plt.plot(real_data, label = 'Real Data')
plt.plot(predict_data, label = 'predicted data')
plt.title('Time series prediction')
plt.legend()
plt.show()